In [1]:
from dynamita.sumo import *

import numpy as np
import time
import copy as cp
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
sumo = Sumo(sumoPath="C:/Users/Sara/AppData/Local/Dynamita/Sumo19",
           licenseFile=r"C:/Users/Sara/Desktop/sewerWRRF/networklicense.sumolic")

License OK...


To load and run a dynamic input:
1. Save the dynamic input(s) in a .tsv file.
  - To make this easy, you can first do it from the Sumo GUI. This will create a .tsv file in the project's temporary directory (View > Directories > Project Directory) and "loadtsv ..." will be logged in the Core Window.
2. Copy the new initialization script and dynamic influent .tsv file into the working folder.
3. Make sure the command "loadtsv ... xxx.tsv" is in the initialization script.
  - If it is not there, find it in the Core Window and paste it into the initialization script (the one in the working folder). It shouldn't matter where it is in the script.
  - Make sure the .tsv file path is updated to the working folder.

In [3]:
def datacomm_callback(sumo):
    t_set.append(sumo.core.csumo_var_get_time_double(sumo.handle))
    
    q_infl_set.append(sumo.core.csumo_var_get_pvt_pos(sumo.handle, q_infl_pos))
    snhx_infl_set.append(sumo.core.csumo_var_get_pvt_pos(sumo.handle, snhx_infl_pos))
    
    q_effl_set.append(sumo.core.csumo_var_get_pvt_pos(sumo.handle, q_effl_pos))
    snhx_effl_set.append(sumo.core.csumo_var_get_pvt_pos(sumo.handle, snhx_effl_pos))
    
    return 0

sumo.unload_model()
sumo.load_model('A2O plant_dynamic.sumo')
    
sumo.core.csumo_command_send(sumo.handle, b'execute script_Initialize_dynamic.scs;')
    
sumo.register_datacomm_callback(datacomm_callback)

def message_callback(sumo):
    for message in sumo.messages:
        print(message)
    sumo.messages = []
    return 0

sumo.register_message_callback(message_callback)

sumo.set_stopTime(10*24*60*60*1000)
sumo.set_dataComm(50*60*1000)

No model is loaded
530021 Set: Sumo__StopTime to 0
530021 Set: Sumo__DataComm to 3600000
530021 Set: Sumo__PlantName to C:\Users\DYNAMI~1\AppData\Local\Dynamita\Sumo18\TMP~1\xjzoiwqs.cba\SUMOPR~1.XML
530049 Core loop started.
530036 Script file script_Initialize_dynamic.scs loaded.
530007 Path set to: "C:/Users/Sara/AppData/Local/Dynamita/Sumo19/.tmp/t0g5mkii.lrm".
530020 Set mode: dynamic
530021 Set: Sumo__Plant__CSTR__param__L_V to 1000
530021 Set: Sumo__Plant__CSTR__param__Qair_N to 0
530021 Set: Sumo__Plant__CSTR2__param__L_V to 1500
530021 Set: Sumo__Plant__CSTR2__param__Qair_N to 0
530021 Set: Sumo__Plant__CSTR3__param__L_V to 4500
530021 Set: Sumo__Plant__Sideflowdivider__param__Qpumped_target to 72000
530030 TSV file "C:\Users\Sara\Desktop\sewerWRRF\SumoPythonPractice\Influent_Table1.tsv" loaded.
530021 Set: Sumo__StopTime to 864000000
530021 Set: Sumo__DataComm to 3000000


In [4]:
# Store positions for variables of interest
q_infl_pos = sumo.core.csumo_model_get_variable_info_pos(sumo.handle, b'Sumo__Plant__Influent__Q')
snhx_infl_pos = sumo.core.csumo_model_get_variable_info_pos(sumo.handle, b'Sumo__Plant__Influent__SNHx')

q_effl_pos = sumo.core.csumo_model_get_variable_info_pos(sumo.handle, b'Sumo__Plant__Effluent__Q')
snhx_effl_pos = sumo.core.csumo_model_get_variable_info_pos(sumo.handle, b'Sumo__Plant__Effluent__SNHx')

In [5]:
t_set = []
q_infl_set = []
snhx_infl_set = []

q_effl_set = []
snhx_effl_set = []

sumo.run_model()

while not sumo.simulation_finished:
    time.sleep(0.01)

530002 Simulation started.
530004 Simulation ended.


In [ ]:
fig, axes = plt.subplots(1,2, figsize=(14,6))

axes[0].set_xlabel('time')
axes[0].set_ylabel('Eff Q')

axes[1].set_xlabel('time')
axes[1].set_ylabel('Eff N')

axes[0].plot(t_set, q_infl_set, label='Q influent')
axes[0].plot(t_set, q_effl_set, label='Q effluent')

axes[1].plot(t_set, snhx_effl_set, label='SNHx')

axes[0].legend(loc='upper right')
axes[1].legend(loc='upper right')

### Editing the dynamic influent .tsv file

In [ ]:
import pandas as pd

In [ ]:
# Specify .tsv for reading and .tsv for writing
r_influentTSV = 'C:/Users/Sara/Desktop/sewerWRRF/SumoPythonPractice/Influent_Table1.tsv'
w_influentTSV = 'C:/Users/Sara/Desktop/sewerWRRF/SumoPythonPractice/Influent_Table1_edit.tsv'

# Read the original .tsv file as a pandas table
r_influentTSV_data = pd.read_table(r_influentTSV, sep='\t')

# Make a change to the original .tsv file
r_influentTSV_data.loc[1]["Sumo__Plant__Influent__param__Q"] = 5000
r_influentTSV_data

# Write the changed .tsv file to a new file
with open(w_influentTSV,'w') as write_tsv:
    write_tsv.write(r_influentTSV_data.to_csv(sep='\t', index=False))

In [ ]:
# Load the changed .tsv file to Sumo
#command = 'set Sumo__Plant__param__Sumo1__muNITO 1.95;'
#command = 'set Sumo__Plant__param__Sumo1__muNITO ' + str(mu) + ';'
#sumo.core.csumo_command_send(sumo.handle, command.encode('utf8'))

command = 'loadtsv '+ w_influentTSV +' ;'
sumo.core.csumo_command_send(sumo.handle, command.encode('utf8'))
#sumo.core.csumo_command_send(sumo.handle, b'loadtsv "C:\Users\Sara\Desktop\sewerWRRF\SumoPythonPractice\Influent_Table1.tsv" ;')

In [ ]:
t_set = []
q_infl_set = []
snhx_infl_set = []

q_effl_set = []
snhx_effl_set = []

sumo.run_model()

while not sumo.simulation_finished:
    time.sleep(0.01)

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(14,6))

axes[0].set_xlabel('time')
axes[0].set_ylabel('Eff Q')

axes[1].set_xlabel('time')
axes[1].set_ylabel('Eff N')

axes[0].plot(t_set, q_infl_set, label='Q influent')
axes[0].plot(t_set, q_effl_set, label='Q effluent')

axes[1].plot(t_set, snhx_effl_set, label='SNHx')

axes[0].legend(loc='upper right')
axes[1].legend(loc='upper right')